### Installing openEO client libraries

To use openEO, you need to make sure that the openEO client libraries are installed. The library is available on pypi, so it can be installed with pip: 
https://pypi.org/project/openeo/

We recommend using at least Python 3.6 so in the notebook environment, the install command is:

In [1]:
!pip3.6 install --user openeo

Looking in indexes: https://artifactory.vgt.vito.be/api/pypi/python-packages/simple
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
## loading of the required functions 

import openeo
import json
import os
import geopandas as gpd
import pandas as pd

### Create openeo session

In [3]:
endpoint = os.environ.get("ENDPOINT", "http://openeo-dev.vgt.vito.be")
openeo_url = "{e}/openeo/{v}".format(e=endpoint, v="1.0.0")
openeo_user= os.environ.get('OPENEO_USER', os.environ['USER'])  #  your Terrascope username
openeo_pass= os.environ.get('OPENEO_PASS', os.environ['USER'] + '123') # replace user with your Terrascope username
### INFO on authentication in Openeo can be found here: https://open-eo.github.io/openeo-python-client/auth.html
session = openeo.connect(openeo_url)
session.authenticate_basic(openeo_user, openeo_pass)

<Connection to 'http://openeo-dev.vgt.vito.be/openeo/1.0.0' with BearerAuth>

### Description on how to use the emergence detection service

### Summary
Predicts the emergence date(s) for each input field in the dataset. 
The detection method is purely based on S1 (Sentinel-1) data. All the possible emergence dates for the given 
time range are provided per field in JSON format. The developed method makes use some of some rules 
one the S1 timeseries profile to derive emergence moments. 


### Required parameters

* **date**: The period for which satellite data should be retrieved and used to estimate emergence date(s)
  * e.g. "2017-01-01","2019-12-31".  
* **polygon**: Featurecollection of polygons for which emergence should be estimated

### Output
The output of the service is a JSON object with all the emergence date(s) that are detected for each field. 
The fields are called in terms of order in the input FeatureCollection 
e.g. Field_0 -> The first field in the input  FeatureCollection



### Disclaimer:
Requesting in NRT (near real-time) is not recommended getting emergence dates.
This is because the emergence date is searched by looking if a peak in backscatter happens after it or not. 
Therefore, one should consider a period of about 2 months after the actual emergence date to get a more robust result.
Furthermore, one should take into account that this method is only based on radar signal so for some croptypes
the result might be less good or during off-season it might be that falsely emergence is detected. 

Also please use as input time range at least a period of about 12 months to get valid results. 

In [4]:
# Define input parameters
 
time_range = "2019-01-31", "2019-12-31"
geom_path = "/data/users/Public/bontek/e_shape/Notebooks/test_fields/test_fields_WIG2019.shp" #the location were the input geometries are stored on the Public Terrascope drive
gpd_shp = gpd.read_file(geom_path)
js_shp = json.loads(gpd_shp.to_json()) # the way the data should be loaded and send to the service     

Emergence_process = session.datacube_from_process("Emergence_detection_S1_only", namespace="vito", date=time_range ,polygon = js_shp) #create the process graph for the service


#print(Emergence_process.graph)


# Obtain result

### option 1 -> When executing for many fields over a long time period (do asynchronous call)
Emergence_result = Emergence_process.send_job().start_and_wait().get_result().load_json() #The result of the emergence prediction will be returned once the job for the service is finished

### option 2 -> When executing for only few fields and for a short time period, a synchronous call can be done which is quite fast

#Emergence_result = Emergence_process.execute()



/home/bontek/.local/lib/python3.6/site-packages/openeo/metadata.py:219: UserWarning: No cube:dimensions metadata
  complain("No cube:dimensions metadata")


0:00:00 Job '6d7593ac-74da-43e8-9393-c85f02d3bb9c': queued (progress N/A)
0:00:05.049947 Job '6d7593ac-74da-43e8-9393-c85f02d3bb9c': queued (progress N/A)
0:00:11.385480 Job '6d7593ac-74da-43e8-9393-c85f02d3bb9c': queued (progress N/A)
0:00:19.244012 Job '6d7593ac-74da-43e8-9393-c85f02d3bb9c': queued (progress N/A)
0:00:29.120852 Job '6d7593ac-74da-43e8-9393-c85f02d3bb9c': queued (progress N/A)
0:00:41.379564 Job '6d7593ac-74da-43e8-9393-c85f02d3bb9c': queued (progress N/A)
0:00:56.730983 Job '6d7593ac-74da-43e8-9393-c85f02d3bb9c': queued (progress N/A)
0:01:15.928817 Job '6d7593ac-74da-43e8-9393-c85f02d3bb9c': queued (progress N/A)
0:01:39.833342 Job '6d7593ac-74da-43e8-9393-c85f02d3bb9c': running (progress N/A)
0:02:09.763573 Job '6d7593ac-74da-43e8-9393-c85f02d3bb9c': running (progress N/A)
0:02:47.086440 Job '6d7593ac-74da-43e8-9393-c85f02d3bb9c': running (progress N/A)
0:03:33.798962 Job '6d7593ac-74da-43e8-9393-c85f02d3bb9c': finished (progress N/A)


### Store the output result



In [5]:
df_emerg_result = pd.DataFrame.from_dict(Emergence_result)
print(df_emerg_result)
df_emerg_result.to_csv('Emerg_result.csv', index = True) # This stores the CSV file in the same folder as the notebook is stored

        Emergence_date_0 Emergence_date_1
Field_0       2019-05-02       2019-09-04
Field_1       2019-05-10       2019-08-22
Field_2       2019-04-27             None
